# Anthropogenic heat from vehicular traffic

This notebook is used to process population data, and ultimately produce the anthropogenic heat flux (AHF) from vehicular traffic for the Greater Los Angeles (LA) region.